In [ ]:
# Import libraries
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, ViTForImageClassification
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

# Parameters
batch_size = 128
max_length = 100
vocab_size = 10000
embed_dim = 200
num_layers = 2
num_heads = 4
ff_dim = 256
num_classes = 2
image_size = 224

# Transformer Encoder Class
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_length, num_layers, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding = nn.Embedding(max_length, embed_dim)
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        positions = torch.arange(x.size(1), device=x.device).unsqueeze(0).expand(x.size(0), -1)
        x = self.embedding(x) + self.position_embedding(positions)
        for layer in self.layers:
            x = layer(x)
        return x

# Dataset Preparation for Text Classification
def prepare_text_data():
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def preprocess_text(text):
        return tokenizer(
            text, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt"
        )

    return preprocess_text

# Vision Transformer Class
class VisionTransformerCls(nn.Module):
    def __init__(self, image_size, embed_dim, num_heads, ff_dim, num_classes, patch_size=16, dropout=0.1):
        super().__init__()
        self.patch_embedding = nn.Conv2d(3, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_embedding = nn.Parameter(torch.randn((image_size // patch_size)**2, embed_dim))
        self.transformer_layer = nn.TransformerEncoderLayer(embed_dim, num_heads, ff_dim, dropout)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.patch_embedding(x).flatten(2).permute(0, 2, 1)
        x += self.positional_embedding
        x = self.transformer_layer(x)
        x = x[:, 0, :]
        return self.fc(x)

# Dataset Preparation for Vision Transformer
def prepare_vision_data():
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    dataset = ImageFolder(root="./flower_photos", transform=transform)
    num_samples = len(dataset)
    train_size = int(num_samples * 0.8)
    val_size = int(num_samples * 0.1)
    test_size = num_samples - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    return train_dataset, val_dataset, test_dataset

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Example Usage
if __name__ == "__main__":
    # Text Classification
    text_preprocessor = prepare_text_data()
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    text_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)
    text_args = TrainingArguments(output_dir="./results", num_train_epochs=3, per_device_train_batch_size=batch_size)

    # Vision Classification
    train_dataset, val_dataset, test_dataset = prepare_vision_data()
    vision_model = VisionTransformerCls(image_size, embed_dim, num_heads, ff_dim, num_classes)
    vision_criterion = nn.CrossEntropyLoss()
    vision_optimizer = Adam(vision_model.parameters(), lr=0.001)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    train_model(vision_model, train_loader, val_loader, vision_criterion, vision_optimizer, num_epochs=5, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
